In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(".."))

from src.data_eng.pipeline import run_pipeline


Create Config

add additional interaction features

In [2]:
from src.config import Config

conf = Config(
    #features=[], all features
    add_int_features=True,
    target={'horizon': 5, 'threshold': 0.01},
   # ticker_list=['AAPL','META'], #all tickers
    validate_cutoff='2022-01-01',      # FINAL TEST START
    fold_len='365D',
    fold_mode='expanding',             # or 'sliding'
    sliding_train_years=None,          # set e.g. 5 if using sliding
    embargo_days=None                  # defaults to horizon=5
)

In [28]:
run_pipeline(conf)

begin fetching data from yfinance...
['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AVGO', 'TSM', 'ORCL', 'WMT', 'JPM', 'INTC', 'UNH', 'HD']
saved: ../data/raw/AAPL.csv
saved: ../data/raw/MSFT.csv
saved: ../data/raw/NVDA.csv
saved: ../data/raw/GOOGL.csv
saved: ../data/raw/AMZN.csv
saved: ../data/raw/META.csv
saved: ../data/raw/TSLA.csv
saved: ../data/raw/AVGO.csv
saved: ../data/raw/TSM.csv
saved: ../data/raw/ORCL.csv
saved: ../data/raw/WMT.csv
saved: ../data/raw/JPM.csv
saved: ../data/raw/INTC.csv
saved: ../data/raw/UNH.csv
saved: ../data/raw/HD.csv
done fetching data
being data cleaning...
done cleaning data
begin feature engineering
make features for AAPL
make features for MSFT
make features for NVDA
make features for GOOGL
make features for AMZN
make features for META
make features for TSLA
make features for AVGO
make features for TSM
make features for ORCL
make features for WMT
make features for JPM
make features for INTC
make features for UNH
make features for HD
Done 

In [3]:
import numpy as np
import pandas as pd


## Hyperparameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from src.data_eng.get_data import get_train_test_data

import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)


X_test, y_test, X_train, y_train = get_train_test_data(conf)

def numeric_only(df: pd.DataFrame) -> pd.DataFrame:
    return df.select_dtypes(include=[np.number]).copy()

X_train = numeric_only(X_train).replace([np.inf, -np.inf], np.nan).fillna(0.0)
X_test  = numeric_only(X_test).replace([np.inf, -np.inf], np.nan).fillna(0.0)



pipe = make_pipeline(
    StandardScaler(with_mean=False),
    LinearSVC(
        max_iter=10000,
        loss="hinge",          # fixed as requested
        dual=True,             # hinge requires dual=True
        class_weight="balanced"  # fixed as requested
    )
)
# params
param_dist = {
    "linearsvc__C": loguniform(1e-3, 1) # reduced search area to low C to speed up
}

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=50,                   
    scoring="roc_auc",           
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

search.fit(X_train, y_train)

print("Best parameters:", search.best_params_)
print("Best score:", search.best_score_)

best_C = search.best_params_['linearsvc__C'] # 0.00115279871282324

best_model = search.best_estimator_


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'linearsvc__C': np.float64(0.00115279871282324)}
Best score: 0.5315366314729787


### Linear SVC

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
# 1) build preprocessor with imputation to kill NaNs from lags/rolls
def make_global_pipeline(numeric_cols, C):
    pre = ColumnTransformer([
        ("num", Pipeline([
            ("impute", SimpleImputer(strategy="median")),
            ("scale", StandardScaler())
        ]), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["__ticker__"]),
    ])
    return Pipeline([
        ("pre", pre),
        ("clf", LinearSVC(
            penalty="l2",
            loss="hinge",           
            dual=True,             
            class_weight="balanced",
            C=C,
            max_iter=20000,         
            random_state=42
        ))
    ])

In [11]:


from sklearn.metrics import roc_auc_score, accuracy_score

from src.data_eng.folds import load_multi_ticker_collection
from src.modeling.global_pairs import build_global_fold_pairs, build_global_insample_and_test

# 1) get data
collection = load_multi_ticker_collection(conf)

# 2) build global fold pairs
pairs = build_global_fold_pairs(collection)

cv_scores = []
for k, (Xtr, ytr, Xva, yva) in enumerate(pairs):
    num_cols = [c for c in Xtr.columns if c not in ("__ticker__", "Date")]
    
    # align drops: build a mask from *X* then apply to X and y
    mask_tr = Xtr[num_cols].isna().any(axis=1)
    Xtr = Xtr.loc[~mask_tr]
    ytr = ytr.loc[~mask_tr]

    mask_va = Xva[num_cols].isna().any(axis=1)
    Xva = Xva.loc[~mask_va]
    yva = yva.loc[~mask_va]

    pipe = make_global_pipeline(num_cols, C=best_C)
    pipe.fit(Xtr, ytr)

    
    scores = pipe.decision_function(Xva)              
    preds  = (scores >= 0).astype(int)                # 0 threshold for hinge

    cv_scores.append({
        "fold": k,
        "roc_auc": roc_auc_score(yva, scores),
        "accuracy": accuracy_score(yva, preds),
        "n_val": len(Xva)
    })

In [12]:
# 4) final test
X_ins, y_ins, X_test, y_test = build_global_insample_and_test(collection)
num_cols = [c for c in X_ins.columns if c not in ("__ticker__", "Date")]

mask_ins  = X_ins[num_cols].isna().any(axis=1)
X_ins     = X_ins.loc[~mask_ins]
y_ins     = y_ins.loc[~mask_ins]

mask_test = X_test[num_cols].isna().any(axis=1)
X_test    = X_test.loc[~mask_test]
y_test    = y_test.loc[~mask_test]

final_pipe = make_global_pipeline(num_cols, C=best_C)
final_pipe.fit(X_ins, y_ins)

final_auc = roc_auc_score(y_test, final_pipe.decision_function(X_test))
final_acc = accuracy_score(y_test, (final_pipe.decision_function(X_test) >= 0).astype(int))
print({"final_auc": final_auc, "final_acc": final_acc})

{'final_auc': 0.5352621760548336, 'final_acc': 0.4633724699272492}


output:
{'final_auc': 0.5352621760548336, 'final_acc': 0.4633724699272492}

This is not great, as expected

### Stochastic Gradient Descent Classifier